In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer

data_base_dir = '../../datasets/Movielens/'
data_dir2 = data_base_dir + 'Movielens Latest/ml-latest/'
data_dir = data_base_dir + 'ml-20m/'

genome_scores = data_dir + 'genome-scores.csv'
genome_tags = data_dir + 'genome-tags.csv'
movies = data_dir + 'movies.csv'
ratings = data_dir + 'ratings.csv'
tags = data_dir + 'tags.csv'

genome_tags_df = pd.read_csv(genome_tags)
genome_tags_df.set_index(genome_tags_df['tagId'].values, drop=True, inplace=True)
genome_tags_df.drop(labels='tagId', axis=1, inplace=True)

genome_scores_df = pd.read_csv(genome_scores)
genome_scores_df = genome_scores_df.pivot(index='movieId', columns='tagId', values='relevance')

ratings_df = pd.read_csv(ratings)

In [7]:
all_user_ids = ratings_df['userId'].unique()
genome_scores_df

tagId,1,2,3,4,5,6,7,8,9,10,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,...,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,...,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,...,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,...,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,...,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825
6,0.02825,0.02550,0.01850,0.04550,0.09575,0.05500,0.04400,0.24200,0.12850,0.02550,...,0.04900,0.01825,0.02075,0.06000,0.29975,0.15525,0.03525,0.01950,0.06650,0.01900
7,0.04575,0.05275,0.16675,0.08275,0.11450,0.15625,0.05025,0.11175,0.03950,0.08000,...,0.03750,0.02825,0.01200,0.03575,0.13000,0.04875,0.01975,0.01050,0.10925,0.01850
8,0.03075,0.03550,0.04675,0.02175,0.05600,0.03650,0.01675,0.07325,0.02950,0.04100,...,0.03700,0.01925,0.01625,0.02325,0.20975,0.02825,0.01675,0.01125,0.07000,0.01500
9,0.03500,0.04050,0.01825,0.01800,0.03650,0.01750,0.01300,0.04225,0.01675,0.01525,...,0.02225,0.01075,0.01175,0.01525,0.14100,0.02225,0.01100,0.00700,0.07275,0.01550


In [15]:
# user=
user_movie_list = ratings_df.loc[:, ['userId', 'movieId']].groupby('userId')['movieId'].apply(list)
user_movie_list

userId
1         [2, 29, 32, 47, 50, 112, 151, 223, 253, 260, 2...
2         [3, 62, 70, 110, 242, 260, 266, 469, 480, 541,...
3         [1, 24, 32, 50, 160, 173, 175, 196, 223, 260, ...
4         [6, 10, 19, 32, 165, 329, 350, 356, 367, 368, ...
5         [2, 11, 17, 60, 62, 104, 110, 140, 141, 150, 2...
6         [1, 3, 7, 17, 52, 62, 135, 140, 141, 260, 494,...
7         [3, 7, 11, 15, 16, 17, 24, 105, 122, 151, 252,...
8         [1, 3, 6, 10, 19, 21, 39, 47, 48, 110, 150, 15...
9         [356, 858, 1219, 1911, 1923, 1997, 2279, 2605,...
10        [1, 11, 25, 260, 356, 527, 858, 912, 969, 1094...
11        [1, 10, 19, 32, 39, 65, 110, 145, 150, 153, 15...
12        [1, 3, 5, 6, 7, 17, 32, 34, 36, 95, 104, 110, ...
13        [1, 2, 10, 11, 19, 32, 34, 39, 47, 95, 110, 15...
14        [1, 7, 17, 34, 62, 150, 252, 260, 277, 314, 31...
15        [10, 11, 14, 17, 21, 34, 36, 39, 50, 141, 150,...
16        [1, 6, 39, 122, 223, 356, 539, 550, 587, 915, ...
17        [6, 110, 377, 457, 480,

In [31]:
from time import time

user_movies_df = pd.DataFrame(user_movie_list)

def process(x):
    return genome_scores_df.loc[x['movieId'], :].mean()

start_time = time()
# user_movies_df = user_movies_df.loc[:10000, :]
user_genome_vec_df = user_movies_df.apply(lambda x: process(x), axis=1)

finish_time = time() - start_time

print("Total time %f seconds" % finish_time)

user_genome_vec_df

Total time 226.418547 seconds


tagId,1,2,3,4,5,6,7,8,9,10,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
userId,,,,,,,,,,,,,,,,,,,,,
1,0.050899,0.040983,0.097598,0.110596,0.162099,0.128846,0.074806,0.210320,0.214312,0.083480,...,0.076131,0.039428,0.051772,0.070019,0.203720,0.095501,0.064954,0.039839,0.143749,0.052318
2,0.058922,0.067475,0.090770,0.089832,0.125361,0.158447,0.081377,0.200102,0.207385,0.066504,...,0.065033,0.053898,0.046852,0.051611,0.212029,0.076389,0.038230,0.047029,0.138463,0.027943
3,0.054747,0.057932,0.071159,0.099066,0.153595,0.163076,0.086672,0.224027,0.238541,0.057223,...,0.080467,0.038194,0.051199,0.058807,0.229066,0.096785,0.035064,0.040388,0.123822,0.045195
4,0.097152,0.081562,0.049411,0.064821,0.109839,0.104009,0.068875,0.172232,0.102223,0.050188,...,0.099911,0.025018,0.023277,0.038259,0.170964,0.062000,0.026643,0.019839,0.077482,0.018196
5,0.048595,0.042864,0.114455,0.085883,0.166686,0.137159,0.086295,0.218701,0.164674,0.083087,...,0.114640,0.034867,0.059398,0.055549,0.183091,0.090337,0.030284,0.039356,0.082295,0.020572
6,0.060969,0.058542,0.087615,0.063469,0.103604,0.108479,0.063698,0.163406,0.094677,0.072458,...,0.067823,0.028906,0.036792,0.037687,0.195583,0.073521,0.025563,0.031229,0.084292,0.018740
7,0.048451,0.049444,0.087064,0.096836,0.164282,0.151078,0.079719,0.177247,0.166044,0.068461,...,0.089058,0.050055,0.063687,0.056028,0.235747,0.107143,0.028797,0.056050,0.088210,0.021103
8,0.067714,0.058054,0.082529,0.066636,0.107275,0.100675,0.060143,0.170364,0.120889,0.050689,...,0.098443,0.047779,0.041146,0.038539,0.192246,0.074332,0.025318,0.040429,0.084868,0.021282
9,0.068757,0.044114,0.029314,0.052450,0.089243,0.089721,0.060650,0.137664,0.116314,0.036000,...,0.059200,0.024271,0.025657,0.039679,0.239479,0.076921,0.027929,0.023157,0.151721,0.056829
